In [1]:
import os
import sys
import numpy
import random
from time import sleep
from copy import deepcopy
import tensorflow.keras as keras
from keras import layers
from itertools import count
from gui.puzzle_class import Twisty_Puzzle
from gui.ai_modules.ai_data_preparation import state_for_ai
from gui.ai_modules.ai_puzzle_class import puzzle_ai
from gui.ai_modules.twisty_puzzle_model import *

def spread_fixed_sorted (l, iterations):
    temp = [l.copy(), l.copy()]
    length = len(l)-1
    curr = 0
    other = 1
    for _ in range(iterations):
        for i in range(1, length):
            temp[other][i] = ((temp[curr][i-1] + temp[curr][i+1])/2 +
                              temp[curr][i])/2
        other, curr = curr, other
    return temp[curr]

def get_sort (l):
    return sorted(range(len(l)), key=lambda k: l[k])

def apply_perm (l, p):
    return [l[s] for s in p]

def sort (l):
    p = get_sort(l)
    return apply_perm(l, p), p

def perm_invert (p):
    out = p.copy()
    for i,j in enumerate(p):
        out[j]=i
    return out

def spread_fixed (l, iterations):
    l, p = sort(l)
    return apply_perm(spread_fixed_sorted(l, len(l)), perm_invert(p))

def spread (l, iterations, minimum, maximum):
    return spread_fixed([minimum]+l+[maximum], iterations)[1:-1]

<IPython.core.display.Javascript object>

In [2]:
def compute_action_map (puzzle):
    return list(puzzle.moves.keys())

def action_num (action_map, action):
    for i, act in enumerate(action_map):
        if action == act :
            return i

def action_from_num (action_map, num):
    return action_map[num]

def prepare_data (action_map, q_table):
    training_table = dict()
    output_len = len(action_map)
    output_template = [0 for _ in range(output_len)]
    for state_action , value in q_table.items() :
        (state , action) = state_action
        inner = training_table.get(state) or output_template.copy()
        inner[action_num(action_map, action)] = value
        training_table[state] = inner
    for state, actions in training_table.items():
        training_table[state] = spread(actions, -1, 1, output_len)
    return list(training_table.keys()) , list(training_table.values())

def import_q_table (filename):
    q_table = dict()
    with open(filename, "r") as file:
        q_table = eval (file.read())
    return q_table

In [3]:
def initialize_nn (puzzle):
    action_map = compute_action_map(puzzle)
    solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
    input_size = len(solved_state)
    output_size = len(action_map)
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_size,)))
    model.add(layers.Dense(input_size, activation='tanh'))
    model.add(layers.Dense((input_size+output_size)/2, activation='tanh'))
    model.add(layers.Dense(output_size, activation='tanh'))
    model.add(layers.Dense(output_size, activation='tanh'))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model, action_map

In [4]:
def train_nn (model, samples, labels, epochs=100, batch_size=30):
    return model.fit(samples, labels, epochs=epochs, batch_size=batch_size,
                     use_multiprocessing=True, verbose=False)

def prepare_nn (puzzle_name, epochs, batch_size=100):
    puzzle = Twisty_Puzzle()
    puzzle.load_puzzle(puzzle_name)
    (model, action_map) = initialize_nn(puzzle)
    q_table = import_q_table(os.path.join("gui", "puzzles", puzzle_name, "Q_table.txt"))
    (states, actions) = prepare_data(action_map, q_table)
    train_nn(model, states, actions, epochs=epochs, batch_size=batch_size)
    model.save(puzzle_name+".nn")
    return model, puzzle, action_map

In [5]:
def nn_get_greedy_move (model, state, action_map):
    predictions = list(model.predict([tuple(state)], use_multiprocessing=True)[0])
    print(predictions)
    max_value = None
    moves = []
    for move, value in enumerate(predictions):
        if (not max_value) or value > max_value:
            moves = [move]
            max_value = value
        elif value == max_value:
            moves.append(move)
    print(moves)
    return action_map[random.choice(moves)]

def nn_get_epsilon_greedy_move (model, state, action_map, epsilon=0.5):
    if random.random() > epsilon:
        return nn_get_greedy_move(model, state, action_map)
    else:
        return random.choice(action_map)

def nn_solve_puzzle (model, state, solved_state, puzzle, action_map, max_moves, epsilon=0):
    moves = []
    while max_moves >= 0:
        if state == solved_state:
            return moves, True
        else :
            move = nn_get_epsilon_greedy_move(model, state, action_map, epsilon=epsilon)
            moves.append(move)
            state = perform_action(state, puzzle.moves[move])
            max_moves -= 1
    return moves, False

# finally test everything

In [ ]:
## {{{ http://code.activestate.com/recipes/577058/ (r2)
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while 1:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return default
        elif choice in valid.keys():
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
                             "(or 'y' or 'n').\n")
## end of http://code.activestate.com/recipes/577058/ }}}

puzzle_name = input("Which puzzle? ")
model = None
puzzle = None
action_map = []
if query_yes_no("Load the existing neural network?"):
    puzzle = Twisty_Puzzle()
    puzzle.load_puzzle(puzzle_name)
    model = keras.models.load_model(puzzle_name+".nn")
    action_map = compute_action_map(puzzle)
else:
    epochs = int(input("How many epochs? "))
    model, puzzle, action_map = prepare_nn(puzzle_name, epochs)

Which puzzle? ivy_cube
Load the existing neural network? [Y/n] n
How many epochs? 10000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
difficulty = int(input("How many moves to twist out from the solved state? "))

puzzle.reset_to_solved()
solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
state = deepcopy(solved_state)
scramble_moves = scramble(state, puzzle.moves, max_moves=difficulty)
start_state = deepcopy(state)

print("solved state:", solved_state)
print("start:       ", state)
(solution, solved) = nn_solve_puzzle(model, state, solved_state, puzzle, action_map, difficulty*2)
print("end:         ", state)
print(solved)
print(solution)

for move in scramble_moves:
    puzzle.perform_move(move)
for move in solution:
    puzzle.perform_move(move)